In [1]:
# All cooridinates are in [row, col]
# end coordinates
t = [829, 347]
# start coordinates
s = [0, 0]
max_slope = 15

# The algorithm does not work on negative values
# This number should represent the lowest negative value and is used to convert all heights to absolute heights
lowerbound = 420
# Represents a number the absolute height is never greater than
upperbound = 2500

# Data from where all the slopes are taken from
height_map = 'FY23_ADC_Height_PeakNearShackleton.csv'
slope_map = 'FY23_ADC_Slope_PeakNearShackleton.csv'

In [2]:
import numpy as np
import math

# Calculates a multiplier to make algorithm run faster
def calculate_distance(row, col):
    return math.sqrt((t[0]-row)**2 + (t[1]-col)**2).__round__(2)
def calculate_distance_mult(row, col):
    ratio = ((calculate_distance(row, col)/calculate_distance(s[0], s[1]))**2).__round__(2)
    if(ratio < .2): ratio = .2
    return ratio

# Creates 'walls' if the slope is greater than or equal to the threshold
# Each non wall has a value that is set to it's absolute height
# Creates the map of height's as well
height_maze = np.zeros((3200, 3200))

final_height_difference = np.zeros((3200, 3200))

# Fill heights in
with open(height_map, 'r') as r:
    col = 0
    row = 0

    for line in r.readlines():
        line = line[0: len(line)-2]

        for value in line.split(','):
            height_maze[row][col] = ((lowerbound+float(value).__round__(0))*calculate_distance_mult(row, col)).__round__(4)
            final_height_difference[row][col] = float(value).__round__(0)
            col+=1
        col = 0
        row += 1

        if(row == 3200): break
        
# Define coordinates that can not be traversed
with open(slope_map, 'r') as r:
    col = 0
    row = 0

    for line in r.readlines():
        # Removes the end line character on the last int
        line = line[0: len(line)-2]

        for value in line.split(','):
            if(float(value) >= max_slope):
                height_maze[row][col] = upperbound

            col+=1
        
        col = 0
        row += 1

        if(row == 3200):
            break

In [3]:
# Each Node has it's own position and the distance too each of the surrounding nodes
class Node:
    def __init__(self, row, col, steps, parent, parentFrom):
        self.row = row
        self.col = col
        # Amount of nodes needed to traverse to get to this node
        self.steps = steps.__round__(2)

        self.parent = parent
        self.parentFrom = parentFrom

        # gets all of the surrounding node distances
        if(row == 0): self.top = upperbound
        else: self.top = height_maze[row-1][col]
        
        if(row == len(height_maze)-1): self.bot = upperbound
        else: self.bot = height_maze[row+1][col]

        if(col == len(height_maze)-1): self.right = upperbound
        else: self.right = height_maze[row][col+1]

        if(col == 0): self.left = upperbound
        else: self.left = height_maze[row][col-1]

        # the children of this node
        self.topChild = None
        self.botChild = None
        self.rightChild = None
        self.leftChild = None

        self.mapSize = len(height_maze)-1

    def addTop(self):
        self.topChild = Node(self.row-1, self.col, self.steps+max(height_maze[self.row-1][self.col]-height_maze[self.row][self.col], 0), self, 'bot')
    def addBot(self):
        self.botChild = Node(self.row+1, self.col, self.steps+max(height_maze[self.row+1][self.col]-height_maze[self.row][self.col], 0), self, 'top')
    def addRight(self):
        self.rightChild = Node(self.row, self.col+1, self.steps+max(height_maze[self.row][self.col+1]-height_maze[self.row][self.col], 0), self, 'left')
    def addLeft(self):
        self.leftChild = Node(self.row, self.col-1, self.steps+max(height_maze[self.row][self.col-1]-height_maze[self.row][self.col], 0), self, 'right')

    # Returns the [least distance direction, distance]
    def leastDistance(self):
        # Criteria to be a least distance node:
        #   1. Can not already be a child of this node
        #   2. Must have a shorter distance to t than any other direction
        #       a. It is automatically a shorter distance if this node already has a child in a certain direction
        #       b. It is automatically a shorter distance if this node's parent came from that direction
        if(self.topChild == None and self.parentFrom != 'top' and self.row != 0 and (self.top <= self.bot or self.botChild != None or self.parentFrom == 'bot') and (self.top <= self.right or self.rightChild != None or self.parentFrom == 'right') and (self.top <= self.left or self.leftChild != None or self.parentFrom == 'left')):
            return ["top", self.top]
        if(self.botChild == None and self.parentFrom != 'bot' and self.row != self.mapSize and (self.bot <= self.top or self.topChild != None or self.parentFrom == 'top') and (self.bot <= self.right or self.rightChild != None or self.parentFrom == 'right') and (self.bot <= self.left or self.leftChild != None or self.parentFrom == 'left')):
            return ["bot", self.bot]
        if(self.rightChild == None and self.parentFrom != 'right' and self.col != self.mapSize and (self.right <= self.bot or self.botChild != None or self.parentFrom == 'bot') and (self.right <= self.top or self.topChild != None or self.parentFrom == 'top') and (self.right <= self.left or self.leftChild != None or self.parentFrom == 'left')):
            return ["right", self.right]
        if(self.leftChild == None and self.parentFrom != 'left' and self.col != 0 and (self.left <= self.bot or self.botChild != None or self.parentFrom == 'bot') and (self.left <= self.right or self.rightChild != None or self.parentFrom == 'right') and (self.left <= self.top or self.topChild != None or self.parentFrom == 'top')):
            return ["left", self.left]
        
        # All nodes have been explored, none to add
        return ["none", upperbound]
    
    # Gives this node a child with the least distance to t
    def extendLeast(self):
        least = self.leastDistance()[0]

        if(least == "top"): 
            self.addTop()
            return self.topChild
        elif(least == "bot"): 
            self.addBot()
            return self.botChild
        elif(least == "right"): 
            self.addRight()
            return self.rightChild
        elif(least == "left"): 
            self.addLeft()
            return self.leftChild
        # if there is no node to add, return an empty node with max distances
        else: 
            node = Node(0, 0, upperbound, None, 'none')
            node.left = upperbound
            node.right = upperbound
            node.top = upperbound
            node.bot = upperbound
            return node
    def __str__(self):
        return f"Height climbed: {self.steps}. Position (row, col): ({self.row}, {self.col})\nChildren distances (top, bot, right, left): ({self.top}, {self.bot}, {self.right}, {self.left})"
    
    # If this object should be stored higher in the priority queue, return true.
    def __lt__(self, other):
        # this distance straight up less than other
        if(self.steps < other.steps): return True
        # other distance straight up less than this distance
        if(self.steps > other.steps): return False

        # both distances must be the same, use the node that takes the least amount of steps to get to
        return (self.steps < other.steps)
    
    def __eq__(self, other):

        if(type(other) == Node):
            return self.row == other.row and self.col == other.col
        else: return False

In [4]:
from queue import PriorityQueue
import numpy as np

external_nodes = PriorityQueue()
Tree = np.zeros((3200, 3200))

def searchTree(node):
    return Tree[node.row][node.col] == 1


In [5]:
# simple check to make sure t is reachable - if it is upperbound then it is known that it is unreachable
# even if it equals 0, it still may be unreachable
height_maze[t[0]][t[1]]

304.4

In [6]:
notFound = True

# Start node
HEAD = Node(s[0], s[1], 0, None, 'none')
external_nodes.put(HEAD)

# Total height traveled
height_traveled = 0

# Contains path: could get rid of but would need to change the display function - no real need
Path = []   
while(notFound):
    # makes sure there are traversable external nodes
    if(len(external_nodes.queue) == 0):
        print('no nodes to visit')
        break
    
    # Get the lowest value node in external_nodes - already know leastNode will not already be in tree
    leastNode = external_nodes.get()
    Tree[leastNode.row][leastNode.col] = 1
    Path.append([leastNode.row,leastNode.col, 1])
    
    # Found the node
    if(leastNode.row == t[0] and leastNode.col == t[1]):
        print(f'found at: ({leastNode.row}, {leastNode.col})')
        node = leastNode
        while(node != None):
            try:
                height_traveled += max(final_height_difference[node.row][node.col]-final_height_difference[node.parent.row][node.parent.col], 0)
            except: pass
            # Store the shortest path
            Path.append([node.row, node.col, 0.5])
            node = node.parent

        print(f'Took: {leastNode.steps} height climbed to get to the destination.')
        # end the search
        notFound = False
        break
    
    # Add new external nodes for the new addition to tree
    # 3 is because there are at most 3 extensions from a node that are not already in the tree
    for i in range(3):
        # Make sure this external node is not already in the tree - do not add if it is in it
        external = leastNode.extendLeast()
        if not (searchTree(external)):
            external_nodes.put(external)
            break

    if(leastNode.parent != None):
        # 2 is because there are at most 2 extensions from a parent node that are not already in the tree
        for i in range(2):
            # Replace the new addition to the tree's parent's external node
            if(leastNode.parent.leastDistance()[1] < upperbound):
                external = leastNode.parent.extendLeast()
                # Make sure this external node is not already in the tree - do not add if it is in it
                if not (searchTree(external)):
                    external_nodes.put(external)
                    break


found at: (829, 347)
Took: 125.18 height climbed to get to the destination.


In [7]:
print(height_traveled)

543.0


In [11]:
from PIL import Image

# Convert to rgb
newTree = np.zeros((len(height_maze), len(height_maze), 3))

# # show the slopes maze
# for row, each in enumerate(height_maze):
#     for col, val in enumerate(each):
#         if(val == upperbound):
#             newTree[row][col] = [255, 255, 255]
# Show the search line
for each in Path:
    # if(each[2] == 1):
    #     newTree[each[0]][each[1]] = [255, 0, 255]
    # else:
    #     newTree[each[0]][each[1]] = [255, 0, 0]

    if(each[2] != 1):
        newTree[each[0]][each[1]] = [255, 0, 0]
        newTree[each[0]-1][each[1]] = [255, 0, 0]
        newTree[each[0]-1][each[1]+1] = [255, 0, 0]
        newTree[each[0]-1][each[1]-1] = [255, 0, 0]
        newTree[each[0]][each[1]-1] = [255, 0, 0]
        newTree[each[0]][each[1]+1] = [255, 0, 0]
        newTree[each[0]+1][each[1]-1] = [255, 0, 0]
        newTree[each[0]+1][each[1]] = [255, 0, 0]
        newTree[each[0]+1][each[1]+1] = [255, 0, 0]



# show the target and start
for row in range(20):
    for col in range(20):
        if(s[0] + 10-row > 0 and s[1] - col + 10 > 0):
            newTree[s[0]-row+10][s[1]-col+10] = [0, 255, 0]
        newTree[t[0]-row+10][t[1]-col+10] = [0, 255, 0]



im = Image.fromarray(np.uint8(newTree))          
im.show()
im.save('ShortestPath.png')